In [1]:
import pyproj
import pandas as pd
import rasterio

def convert_coordinates(df, source_crs, target_crs):
    """
    Convert coordinates from the source CRS to the target CRS using pyproj.Transformer.

    Args:
        df (pd.DataFrame): DataFrame containing 'latitude' and 'longitude' columns.
        source_crs (str): Source Coordinate Reference System (CRS) in EPSG format.
        target_crs (str): Target Coordinate Reference System (CRS) in EPSG format.

    Returns:
        pd.DataFrame: DataFrame with converted coordinates.
    """
    # Create a transformer from the source to the target CRS
    transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
    
    # Transform the coordinates and update the DataFrame
    converted_coordinates = transformer.transform(df['longitude'].to_numpy(), df['latitude'].to_numpy())
    df['latitude'], df['longitude'] = converted_coordinates[1], converted_coordinates[0]
    
    return df

def process_tiff_files(path, tiff_files):
    """
    Process multiple TIFF files to create a DataFrame with coordinates and intensities.

    Args:
        path (str): Path to the directory containing TIFF files.
        tiff_files (list): List of TIFF file names.

    Returns:
        pd.DataFrame: DataFrame with coordinates and intensities.
        str: Source Coordinate Reference System (CRS).
    """
    data = []

    # Process the first file to establish the grid (coordinates)
    with rasterio.open(path + tiff_files[0]) as dataset:
        source_crs = dataset.crs
        band1 = dataset.read(1)
        for row in range(dataset.height):
            for col in range(dataset.width):
                x, y = dataset.transform * (col, row)
                intensity = band1[row, col]
                data.append({'latitude': y, 'longitude': x, tiff_files[0]: intensity})

    # Process the remaining files
    for file in tiff_files[1:]:
        with rasterio.open(path + file) as dataset:
            band1 = dataset.read(1)
            for i, (row, col) in enumerate([(row, col) for row in range(dataset.height) for col in range(dataset.width)]):
                intensity = band1[row, col]
                data[i][file] = intensity

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)
    
    return df, source_crs

map1_tiff_files = ['B01.tiff', 'B02.tiff', 'B03.tiff', 'B04.tiff', 'B05.tiff', 'B06.tiff', 'B07.tiff', 'B08.tiff', 'B09.tiff', 'B10.tiff', 'B11.tiff', 'B12.tiff', 'B8A.tiff']
map1_tiff_path = 'data/map1/tiff/'

map1_df, source_crs = process_tiff_files(map1_tiff_path, map1_tiff_files)

#save the dataframe to a csv file
map1_df.to_csv('data/map1/raw/map1.csv', index=False)

map1_df

,latitude,longitude,B01.tiff,B02.tiff,B03.tiff,B04.tiff,B05.tiff,B06.tiff,B07.tiff,B08.tiff,B09.tiff,B10.tiff,B11.tiff,B12.tiff,B8A.tiff
0,-23.187776,-54.443661,0.1215,0.1014,0.1098,0.0937,0.1352,0.2671,0.3341,0.3156,0.0432,0.0014,0.2399,0.1266,0.3680
1,-23.187776,-54.443571,0.1217,0.0988,0.1050,0.0827,0.1349,0.2712,0.3395,0.3166,0.0433,0.0014,0.2394,0.1232,0.3726
2,-23.187776,-54.443481,0.1219,0.0983,0.1060,0.0823,0.1347,0.2784,0.3473,0.3221,0.0433,0.0014,0.2421,0.1233,0.3795
3,-23.187776,-54.443391,0.1221,0.0997,0.1073,0.0863,0.1346,0.2869,0.3555,0.3288,0.0432,0.0014,0.2468,0.1252,0.3869
4,-23.187776,-54.443302,0.1223,0.1001,0.1097,0.0872,0.1359,0.2915,0.3587,0.3394,0.0430,0.0014,0.2509,0.1278,0.3904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59747,-23.203689,-54.416430,0.1262,0.1031,0.1086,0.1144,0.1451,0.2792,0.3416,0.3171,0.0475,0.0014,0.3527,0.2176,0.3764
59748,-23.203689,-54.416340,0.1262,0.1037,0.1085,0.1130,0.1448,0.2796,0.3412,0.3147,0.0474,0.0014,0.3510,0.2157,0.3762
59749,-23.203689,-54.416250,0.1261,0.1039,0.1074,0.1100,0.1440,0.2811,0.3428,0.3179,0.0474,0.0014,0.3496,0.2140,0.3776
59750,-23.203689,-54.416160,0.1261,0.1032,0.1073,0.1071,0.1428,0.2838,0.3466,0.3252,0.0473,0.0014,0.3482,0.2124,0.3808


In [2]:
# MAP 2
target_crs = "EPSG:4326"

map2_tiffs_path = 'data/map2/tiff/'
map2_tiffs = ["B01.tiff", "B02.tiff", "B03.tiff", "B04.tiff", "B05.tiff", "B06.tiff", "B07.tiff", "B08.tiff", "B8A.tiff", "B09.tiff", "B11.tiff", "B12.tiff"]

map2_df, source_crs = process_tiff_files(map2_tiffs_path, map2_tiffs)
map2_df = convert_coordinates(map2_df, source_crs=source_crs, target_crs=target_crs)

map2_df.to_csv('data/map2/raw/map2.csv', index=False)

map2_df

,latitude,longitude,B01.tiff,B02.tiff,B03.tiff,B04.tiff,B05.tiff,B06.tiff,B07.tiff,B08.tiff,B8A.tiff,B09.tiff,B11.tiff,B12.tiff
0,-22.314065,-52.681349,0.0565,0.0600,0.0672,0.0494,0.0472,0.0439,0.0445,0.0433,0.0420,0.0389,0.0412,0.0361
1,-22.314065,-52.681259,0.0567,0.0587,0.0651,0.0470,0.0473,0.0438,0.0443,0.0413,0.0419,0.0392,0.0412,0.0360
2,-22.314065,-52.681170,0.0569,0.0591,0.0630,0.0473,0.0484,0.0441,0.0450,0.0416,0.0430,0.0395,0.0416,0.0368
3,-22.314065,-52.681080,0.0569,0.0601,0.0647,0.0491,0.0495,0.0445,0.0458,0.0432,0.0442,0.0396,0.0420,0.0376
4,-22.314065,-52.680990,0.0569,0.0611,0.0662,0.0489,0.0494,0.0454,0.0458,0.0433,0.0448,0.0396,0.0419,0.0376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59747,-22.330080,-52.654118,0.0159,0.0156,0.0411,0.0178,0.0779,0.2690,0.3334,0.3256,0.3600,0.3634,0.1446,0.0603
59748,-22.330080,-52.654028,0.0155,0.0173,0.0419,0.0181,0.0826,0.2811,0.3436,0.3282,0.3718,0.3694,0.1464,0.0609
59749,-22.330080,-52.653939,0.0154,0.0181,0.0439,0.0185,0.0864,0.3009,0.3636,0.3497,0.3917,0.3732,0.1499,0.0616
59750,-22.330080,-52.653849,0.0157,0.0175,0.0507,0.0216,0.0897,0.3038,0.3634,0.3724,0.3916,0.3740,0.1527,0.0634


In [3]:
# %pip install pyproj
# %pip install rasterio
# %pip install pandas